# KDD-CUP99
*intro*

In [1]:
import os
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from time import time
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
y_field = 'y:label'
data_file_name = '../data/kddcup.data_10_percent.gz'
data_file_name = '../data/kddcup.data.gz'

In [3]:
features_types = ["duration: continuous.", "protocol_type: symbolic.", "service: symbolic.", "flag: symbolic.", "src_bytes: continuous.", "dst_bytes: continuous.", "land: symbolic.", "wrong_fragment: continuous.", "urgent: continuous.", "hot: continuous.", "num_failed_logins: continuous.", "logged_in: symbolic.", "num_compromised: continuous.", "root_shell: continuous.", "su_attempted: continuous.", "num_root: continuous.", "num_file_creations: continuous.", "num_shells: continuous.", "num_access_files: continuous.", "num_outbound_cmds: continuous.", "is_host_login: symbolic.", "is_guest_login: symbolic.", "count: continuous.", "srv_count: continuous.", "serror_rate: continuous.", "srv_serror_rate: continuous.", "rerror_rate: continuous.", "srv_rerror_rate: continuous.", "same_srv_rate: continuous.", "diff_srv_rate: continuous.", "srv_diff_host_rate: continuous.", "dst_host_count: continuous.", "dst_host_srv_count: continuous.", "dst_host_same_srv_rate: continuous.", "dst_host_diff_srv_rate: continuous.", "dst_host_same_src_port_rate: continuous.", "dst_host_srv_diff_host_rate: continuous.", "dst_host_serror_rate: continuous.", "dst_host_srv_serror_rate: continuous.", "dst_host_rerror_rate: continuous.", "dst_host_srv_rerror_rate: continuous.",'y:label']
df = pd.read_csv(data_file_name, names=features_types)
print(df.shape)
df.head()

(4898431, 42)


,duration: continuous.,protocol_type: symbolic.,service: symbolic.,flag: symbolic.,src_bytes: continuous.,dst_bytes: continuous.,land: symbolic.,wrong_fragment: continuous.,urgent: continuous.,hot: continuous.,...,dst_host_srv_count: continuous.,dst_host_same_srv_rate: continuous.,dst_host_diff_srv_rate: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_srv_diff_host_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_srv_serror_rate: continuous.,dst_host_rerror_rate: continuous.,dst_host_srv_rerror_rate: continuous.,y:label
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


**Treat as binary problem**

In [4]:
df['y_binary'] = 'benign'
df.loc[df[y_field]!='normal.','y_binary']='malware'
y_field = 'y_binary'
n_classes = df[y_field].nunique()
df['y_binary'].value_counts()

malware    3925650
benign      972781
Name: y_binary, dtype: int64

**numeric vs categorical features**

In [5]:
features_continuous = [x for x in features_types if 'continuous' in x]
features_categorical = ['protocol_type: symbolic.','service: symbolic.','flag: symbolic.']
print(f"#continous features: {len(features_continuous)}, #categorical features: {len(features_categorical)}")

#continous features: 34, #categorical features: 3


**add dummy features for categorical (one hot encoding)**

In [6]:
df = pd.get_dummies(df,columns=features_categorical)
df.shape

(4898431, 124)

**get all new dummy features **

In [7]:
_remove_learn_featrues_all =['service: symbolic._aol','service: symbolic._harvest','service: symbolic._http_2784','service: symbolic._http_8001']
_remove_learn_featrues_10percent = ['service: symbolic._red_i','service: symbolic._urh_i']
features_categorical_extended = [x for y in features_categorical for x in df.columns if x.split('_')[0]==y]
features_categorical_extended = list(set(features_categorical_extended)-set(_remove_learn_featrues_10percent))
if data_file_name =='../data/kddcup.data.gz':    
    features_categorical_extended = list(set(features_categorical_extended)-set(_remove_learn_featrues_all))
# TEST FEATURES != LEARN FEATURES
len(features_categorical_extended)

75

In [8]:
print(df.shape,df.drop_duplicates().shape)

(4898431, 124) (1074992, 124)


In [9]:
# df.drop_duplicates(inplace=True)

## Validate and learn

In [10]:
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,KFold
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,precision_score,classification_report,accuracy_score
from sklearn.feature_selection import SelectFromModel

In [11]:
# Class that identifies Column type
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, names=None):
        self.names = names
    def fit (self, X, y=None, **fit_params):
        return self
    def transform(self, X):
        return X[self.names]
    
class Logger(BaseEstimator):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self,X, y=None):
        print("process done: {}".format(X.shape))
        return X

In [12]:
features = FeatureUnion(
    transformer_list=[
        ("numeric",Pipeline(
        [
            ('selector',ItemSelector(names=features_continuous)),
            ('scaler',MinMaxScaler()),
            ('log',Logger()),
        ])
        ),
        
        
         ("categorical",Pipeline(
        [
            ('selector',ItemSelector(names=features_categorical_extended)),
            ('log',Logger()),
        ])
        ),
    ]
)

In [31]:
# cls = SVC()
# cls = RandomForestClassifier(n_jobs=10, max_depth=5, n_estimators=10, max_features=1)
cls = LogisticRegression()
simple_pipeline = Pipeline(
[
    ('features',features),
    ('select_features',SelectFromModel(LinearSVC())),
    ('log',Logger()),
    ('cls',cls)
],
)

In [32]:
%%notify
t = time()
cv = StratifiedKFold(n_splits=2, shuffle=True)
# cv = StratifiedShuffleSplit(n_splits=3,test_size=0.1)
cms = np.ndarray((cv.n_splits, n_classes * n_classes), int)
for i, (train, test) in zip(range(cv.n_splits),cv.split(df,df[y_field])):
    simple_pipeline.fit(df.iloc[train],df.iloc[train][y_field])
    y_pred = simple_pipeline.predict(df.iloc[test])
    cm = confusion_matrix(df.iloc[test][y_field], y_pred)
#     print(cm)
    print("p_micro:{},p_macro:{}".format(precision_score(df.iloc[test][y_field],y_pred,average='micro'),
                                        precision_score(df.iloc[test][y_field],y_pred,average='macro')))
    print(classification_report(df.iloc[test][y_field],y_pred))
    cms[i] = cm.flatten()
avg_cms = cms.mean(axis=0).reshape((n_classes, n_classes)).astype(int)
print('average confusion matrix:\n{}'.format(avg_cms))
print('estimation finished. Elapsed: {}'.format(time() - t))

process done: (2449215, 34)
process done: (2449215, 75)


D:\MyProgramFiles\Anaconda3\envs\web-page\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


process done: (2449215, 43)
process done: (2449216, 34)
process done: (2449216, 75)
process done: (2449216, 43)
p_micro:0.9977204950482114,p_macro:0.9952736184816483
             precision    recall  f1-score   support

     benign       0.99      1.00      0.99    486391
    malware       1.00      1.00      1.00   1962825

avg / total       1.00      1.00      1.00   2449216

process done: (2449216, 34)
process done: (2449216, 75)


D:\MyProgramFiles\Anaconda3\envs\web-page\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


process done: (2449216, 43)
process done: (2449215, 34)
process done: (2449215, 75)
process done: (2449215, 43)
p_micro:0.9977629567024536,p_macro:0.9954051988571913
             precision    recall  f1-score   support

     benign       0.99      1.00      0.99    486390
    malware       1.00      1.00      1.00   1962825

avg / total       1.00      1.00      1.00   2449215

average confusion matrix:
[[ 485098    1292]
 [   4238 1958586]]
estimation finished. Elapsed: 301.3200874328613


<IPython.core.display.Javascript object>

**Learn on all data**

In [33]:
simple_pipeline.fit(df,df[y_field])

process done: (4898431, 34)
process done: (4898431, 75)


D:\MyProgramFiles\Anaconda3\envs\web-page\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


process done: (4898431, 43)


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('numeric', Pipeline(memory=None,
     steps=[('selector', ItemSelector(names=['duration: continuous.', 'src_bytes: continuous.', 'dst_bytes: continuous.', 'wrong_fragment: continuous.', 'urgent: continuous.', 'hot: continuous.', 'n...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

**Select features**

In [34]:
feature_importances = pd.DataFrame(simple_pipeline.named_steps['cls'].feature_importances_,
                                   index = df[features_continuous + features_categorical_extended].columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

# Estimation

In [35]:
df_test = pd.read_csv('../data/corrected.gz',names=features_types)
print(df_test.shape)
df_test.head()

(311029, 42)


,duration: continuous.,protocol_type: symbolic.,service: symbolic.,flag: symbolic.,src_bytes: continuous.,dst_bytes: continuous.,land: symbolic.,wrong_fragment: continuous.,urgent: continuous.,hot: continuous.,...,dst_host_srv_count: continuous.,dst_host_same_srv_rate: continuous.,dst_host_diff_srv_rate: continuous.,dst_host_same_src_port_rate: continuous.,dst_host_srv_diff_host_rate: continuous.,dst_host_serror_rate: continuous.,dst_host_srv_serror_rate: continuous.,dst_host_rerror_rate: continuous.,dst_host_srv_rerror_rate: continuous.,y:label
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


In [36]:
df_test.drop_duplicates(inplace=True)

In [37]:
df_test['y_binary'] = 'benign'
df_test.loc[df_test['y:label']!='normal.','y_binary']='malware'
y_field = 'y_binary'
df_test = pd.get_dummies(df_test,columns=features_categorical)
df_test['y_binary'].value_counts()

benign     47913
malware    29378
Name: y_binary, dtype: int64

In [38]:
df_test['y_pred'] = simple_pipeline.predict(df_test)
print(classification_report(df_test[y_field],df_test['y_pred']))
print(f"acc:{accuracy_score(df_test[y_field],df_test['y_pred'])}")
print(confusion_matrix(df_test[y_field],df_test['y_pred']))

process done: (77291, 34)
process done: (77291, 75)


D:\MyProgramFiles\Anaconda3\envs\web-page\lib\site-packages\sklearn\utils\__init__.py:93: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(mask.dtype, np.int):


process done: (77291, 43)
             precision    recall  f1-score   support

     benign       0.88      0.98      0.93     47913
    malware       0.97      0.78      0.86     29378

avg / total       0.91      0.91      0.90     77291

acc:0.9071043200372618
[[47128   785]
 [ 6395 22983]]


# Замечания и дальнейшее развитие:
* Можно попробовать поиск оптимальных значений гипер-параметров моделей с помощью gridsearch(cv)
* Попробовать уменьшить размерность признаков с помощью PCA (визуализировать с помощью TSNE)
* Если качество не устраивает, то пробовать комбинировать классификаторы (ансамбли, бустинги)
* Попробовать нейронные сети
* Согласно авторам [статьи](https://www.ee.ryerson.ca/~bagheri/papers/cisda.pdf) не корректно оценивать качество моделей на предлагаемом тестовом датасете, соответственно, попробовать оценить предлагаемую модель на другом [датасете](https://github.com/defcom17/NSL_KDD)

результат на тестовом сете(без удаления дубликатов) с помощью `simple_pipeline`:
```python 
          precision    recall  f1-score   support

     benign       0.88      0.98      0.93     47913
    malware       0.97      0.78      0.86     29378

avg / total       0.91      0.91      0.90     77291

acc:0.9071043200372618
[[47128   785]
 [ 6395 22983]]

simple_pipeline = Pipeline(
[
    ('features',FeatureUnion(
    transformer_list=[
        ("numeric",Pipeline(
        [
            ('selector',ItemSelector(names=features_continuous)),
            ('scaler',MinMaxScaler()),
            ('log',Logger()),
        ])
        ),
         ("categorical",Pipeline(
        [
            ('selector',ItemSelector(names=features_categorical_extended)),
            ('log',Logger()),
        ])
        ),
    ]
)),
    ('select_features',SelectFromModel(LinearSVC())),
    ('log',Logger()),
    ('cls',LogisticRegression())
],
)`
```
